#### Generic Models

Pydantic supports the creation of generic models to make it easier to reuse a common model structure.

In order to declare a generic model, you perform the following steps:

* Declare one or more `typing.TypeVar` instances to use to parameterize your model.

* Declare a pydantic model that inherits from `pydantic.generics.GenericModel` and `typing.Generic`, where you pass the `TypeVar` instances as parameters to `typing.Generic`.

* Use the `TypeVar` instances as annotations where you will want to replace them with other types or pydantic models.

Here is an example using `GenericModel` to create an easily-reused HTTP response payload wrapper.

In [1]:
from typing import Generic, TypeVar, Optional, List, Any, Type, Tuple

from pydantic import BaseModel, validator, ValidationError
from pydantic.generics import GenericModel

In [2]:
DataT = TypeVar("DataT")

In [3]:
class Error(BaseModel):
    code: int
    message: str

In [4]:
class DataModel(BaseModel):
    numbers: List[int]
    people: List[str]

In [5]:
class Response(GenericModel, Generic[DataT]):
    data: Optional[DataT]
    error: Optional[Error]

    @validator("error", always=True)
    def check_consistency(cls, v, values):
        if v is not None and values["data"] is not None:
            raise ValueError("must not provide both data and error")
        if v is None and values.get("data") is None:
            raise ValueError("must provide data or error")
        return v

In [6]:
data = DataModel(numbers=[1, 2, 3], people=[])
error = Error(code=404, message="Not found")

In [7]:
print(Response[int](data=1))

data=1 error=None


In [8]:
print(Response[str](data="value"))

data='value' error=None


In [9]:
print(Response[str](data="value").dict())

{'data': 'value', 'error': None}


In [10]:
print(Response[DataModel](data=data).dict())

{'data': {'numbers': [1, 2, 3], 'people': []}, 'error': None}


In [11]:
print(Response[DataModel](error=error).dict())

{'data': None, 'error': {'code': 404, 'message': 'Not found'}}


In [12]:
try:
    print(Response[int](data="value"))
except ValidationError as e:
    print(e)

2 validation errors for Response[int]
data
  value is not a valid integer (type=type_error.integer)
error
  must provide data or error (type=value_error)


If you set `Config` or make use of `validator` in your generic model definition, it is applied to concrete subclasses in the same way as when inheriting from `BaseModel`. Any methods defined on your generic class will also be inherited.

Pydantic's generics also integrate properly with mypy, so you get all the type checking you would expect mypy to provide if you were to declare the type without using `GenericModel`.

> ##### Note
> 
> Internally, pydantic uses create_model to generate a (cached) concrete BaseModel at runtime, so there is essentially zero overhead introduced by making use of GenericModel.

To inherit from a GenericModel without replacing the `TypeVar` instance, a class must also inherit from `typing.Generic`.

In [13]:
TypeX = TypeVar("TypeX")

In [14]:
class BaseClass(GenericModel, Generic[TypeX]):
    X: TypeX

In [15]:
class ChildClass(BaseClass[TypeX], Generic[TypeX]):
    pass

In [16]:
print(ChildClass[int](X=1))

X=1


You can also create a generic subclass of a `GenericModel` that partially or fully replaces the type parameters in the superclass.

In [17]:
TypeY = TypeVar("TypeY")
TypeZ = TypeVar("TypeZ")

In [18]:
class GenericBaseClass(GenericModel, Generic[TypeX, TypeY]):
    x: TypeX
    y: TypeY

In [19]:
class GenericChildClass(GenericBaseClass[int, TypeY], Generic[TypeY, TypeZ]):
    z: TypeZ

In [20]:
print(GenericChildClass[str, int](x=1, y="y", z=3))

x=1 y='y' z=3


If the name of the concrete subclasses is important, you can also override the default behavior.

In [21]:
class DataResponse(GenericModel, Generic[DataT]):
    data: DataT

    @classmethod
    def __concrete_name__(cls: Type[Any], params: Tuple[Type[Any], ...]) -> str:
        return f"{params[0].__name__.title()}DataResponse"

In [22]:
print(repr(DataResponse[int](data=1)))
print(repr(DataResponse[str](data='a')))

IntDataResponse(data=1)
StrDataResponse(data='a')


Using the same TypeVar in nested models allows you to enforce typing relationships at different points in your model.

In [23]:
T = TypeVar("T")

In [24]:
class InnerT(GenericModel, Generic[T]):
    inner: T

In [25]:
class OuterT(GenericModel, Generic[T]):
    outer: T
    nested: InnerT[T]

In [26]:
nested = InnerT[int](inner=1)
print(OuterT[int](outer=1, nested=nested))

outer=1 nested=InnerT[int](inner=1)


In [27]:
try:
    nested = InnerT[str](inner="a")
    print(OuterT[int](outer="a", nested=nested))
except ValidationError as e:
    print(e)

2 validation errors for OuterT[int]
outer
  value is not a valid integer (type=type_error.integer)
nested -> inner
  value is not a valid integer (type=type_error.integer)


Pydantic also treats `GenericModel` similarly to how it treats built-in generic types like `List` and `Dict` when it comes to leaving them unparameterized, or using bounded `TypeVar` instances:

* If you don't specify parameters before instantiating the generic model, they will be treated as `Any`

* You can parametrize models with one or more bounded parameters to add subclass checks

Also, like `List` and `Dict`, any parameters specified using a `TypeVar` can later be substituted with concrete types.

In [28]:
AT = TypeVar("AT")
BT = TypeVar("BT")

In [29]:
class Model(GenericModel, Generic[AT, BT]):
    a: AT
    b: BT

In [30]:
print(Model(a="a", b="a"))

a='a' b='a'


In [31]:
IntT = TypeVar("IntT", bound=int)
typevar_model = Model[int, IntT]
print(typevar_model(a=1, b=1))

a=1 b=1


In [32]:
try:
    typevar_model(a="a", b="a")
except ValidationError as exc:
    print(exc)

2 validation errors for Model[int, IntT]
a
  value is not a valid integer (type=type_error.integer)
b
  value is not a valid integer (type=type_error.integer)


In [33]:
concrete_model = typevar_model[int]
print(concrete_model(a=1, b=1))

a=1 b=1
